In [1]:
import torch
ip_adapter_image_embeds = [torch.randn(1,257,1280),torch.randn(1,1,512)]
repeat_dims = [1]
image_embeds = []
num_images_per_prompt = 1
for single_image_embeds in ip_adapter_image_embeds:
    single_image_embeds = single_image_embeds.repeat(
        num_images_per_prompt, *(repeat_dims * len(single_image_embeds.shape[1:]))
    )
    image_embeds.append(single_image_embeds)
image_embeds[0].shape


torch.Size([1, 257, 1280])

In [2]:
image_embeds[1].shape

torch.Size([1, 1, 512])

In [ ]:
import torch
ip_adapter_image_embeds = torch.randn(2, 3, 4)

In [3]:
MATCHED_WORDS = ["person", "male", "female", "man", "woman", "men", "women"]

def insert_img_after_keyword(text):
    for word in MATCHED_WORDS:
        index = text.find(word)
        if index != -1:
            # Find the end index of the matched word
            end_index = index + len(word)
            # Insert "img" after the matched word
            new_text = text[:end_index] + " img" + text[end_index:]
            return new_text
    return text

# Example usage
input_text = "A man wears eyeglasses and hat. This man has beard."
output_text = insert_img_after_keyword(input_text)
print(output_text)

A man img wears eyeglasses and hat. This man has beard.


In [ ]:
import numpy as np
start_idx = 0
video_length = 16  
required_frame_num = 16*8
frame_stride=8
frame_indices = np.linspace(start_idx, start_idx + required_frame_num - 1, num=video_length, dtype=int)
frame_indices = [start_idx + frame_stride*i for i in range(video_length)]

In [1]:
import torch

# 假设我们有两个形状相同的张量
a = torch.tensor([1, 2])
b = torch.tensor([3, 4])

# 使用torch.cat进行拼接
cat_result = torch.cat((a, b), dim=0)  # 结果: tensor([1, 2, 3, 4])

# 使用torch.stack进行堆叠
stack_result = torch.stack((a, b))     # 结果: tensor([[1, 2],
cat_result.shape

torch.Size([4])

In [1]:
import torch
latent = torch.load("/group/40034/jackeywu/code/PhotoMaker/datasets/CeleV-Text/processed_512/___5yD2BVx8_4_0.pt",map_location='cpu')
latent = latent['latent']
latent.shape

torch.Size([4, 16, 64, 64])

In [2]:
latent=latent.unsqueeze(0)
latent.shape

torch.Size([1, 4, 16, 64, 64])

In [3]:
from diffusers import AutoencoderKL
vae = AutoencoderKL.from_pretrained("./pretrain_model/RealVisXL_V4.0", subfolder="vae",use_safetensors=True)
vae = vae.to("cuda")
vae.enable_slicing()
vae.enable_tiling()

/data/miniconda3/envs/env-novelai/lib/python3.10/site-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)


In [4]:
def decode_latents(vae, latents):
    latents = 1 / vae.config.scaling_factor * latents

    batch_size, channels, num_frames, height, width = latents.shape
    latents = latents.permute(0, 2, 1, 3, 4).reshape(batch_size * num_frames, channels, height, width)

    image = vae.decode(latents).sample
    video = image[None, :].reshape((batch_size, num_frames, -1) + image.shape[2:]).permute(0, 2, 1, 3, 4)
    # we always cast to float32 as this does not cause significant overhead and is compatible with bfloat16
    video = video.float()
    return video

In [5]:
from diffusers.video_processor import VideoProcessor
video_processor = VideoProcessor(vae_scale_factor=2 ** (len(vae.config.block_out_channels) - 1))

In [6]:
latent = latent.to("cuda")
video_tensor = decode_latents(vae,latent)
video = video_processor.postprocess_video(video=video_tensor, output_type='pil')
video = video.frames[0]

OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB. GPU 0 has a total capacty of 39.59 GiB of which 182.19 MiB is free. Process 190256 has 39.41 GiB memory in use. Of the allocated memory 37.30 GiB is allocated by PyTorch, and 410.39 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
from diffusers.utils import export_to_gif, load_image
export_to_gif(video, "video.gif")